In [3]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt 
import seaborn
import statsmodels.api as sm
from scipy.stats import zscore
from statsmodels import regression
from statsmodels.tsa.stattools import coint

In [156]:
class Pair_trading():
    def __init__(self, data_var,data_preco, par, valor_inicial):
        self.financeiro = valor_inicial
        self.posicionado = 0 #comeca sem nenhuma operação aberta
        self.lista_resultado_operacoes = []
        self.lista_evolucao_carteira = []
        self.lista_dias_carregando = []
        self.lista_rentabilidade_operacao = []
        self.contador_dias = 0 
        self.perm_pct_acumulado = 1
        #guarda tickers das acoes
        self.t1 = par[0] 
        self.t2 = par[1]
        self.volume_operacao = 0

        self.dados_var = data_var
        self.dados_preco = data_preco

    def calcula_zscore_spread(self,stock1,stock2):
        stock1 = sm.add_constant(stock1)
        results = sm.OLS(stock2,stock1).fit()
        stock1 = stock1[self.t1]
        b = results.params[self.t1]
        spread = stock2 - (b*stock1)
        zs = zscore(spread)[-1]
        return zs
        
    def condicoes_operar(self,zs, gatilho_high, preco_s1, preco_s2, index):
        if zs >= gatilho_high and self.posicionado == 0:
            #parametros para iniciar operacao
            self.posicionado = 1 #entrou em uma operacao            
            self.volume = (self.financeiro/2)//preco_s1
            self.razao = preco_s1/preco_s2
            self.contador_dias = index
            #guarda dados de quantas ações está comprado/vendido
            self.qtd_s1 =  self.volume
            self.qtd_s2 =  self.volume*self.razao
            #calcula valor total da compra e venda (idealmente devem ser iguais)
            valor_operado_s1 =  preco_s1*self.qtd_s1 #vende  s1 (credita caixa)
            valor_operado_s2 = -preco_s2*self.qtd_s2 #compra s2 (debita caixa) 
#             print(f'v1  R${valor_operado_s1:.2f} | v2 R${valor_operado_s2:.2f}')
#             print(f'C1  R${preco_s1:.2f} | D2 R${(preco_s2*self.razao):.2f}')
            self.financeiro += valor_operado_s1 + valor_operado_s2

        elif zs <= 0.08 and self.posicionado == 1:
            #calcula preço das açoes que estão em carteira
            valor_operado_s1 = -self.qtd_s1*preco_s1 #compra s1 (debita  caixa) 
            valor_operado_s2 =  self.qtd_s2*preco_s2 #vende  s2 (credita caixa)
#             print(f'v1 R${valor_operado_s1:.2f} | v2  R${valor_operado_s2:.2f}')
#             print(f'D1  R${preco_s1:.2f} | C2 R${(preco_s2*self.razao):.2f}')
            resultado_trade = valor_operado_s1 + valor_operado_s2 
#             print(f'resu: R${resultado_trade:.2f}\n')
            self.financeiro += resultado_trade 
            #zera quantidades de ações sendo operadas
            self.qtd_s1 -= self.qtd_s1
            self.qtd_s2 -= self.qtd_s2
            #guarda dados para acompanhamento ao longo do tempo
            pct_change_trade = resultado_trade/(self.financeiro-resultado_trade)
            self.perm_pct_acumulado = self.perm_pct_acumulado*(1+pct_change_trade)
            self.lista_rentabilidade_operacao.append(pct_change_trade*100)
            self.lista_evolucao_carteira.append(self.financeiro)
            self.lista_dias_carregando.append(index-self.contador_dias)
            self.lista_resultado_operacoes.append(resultado_trade)
            #zera parametros para próximas operações 
            self.posicionado = 0
            self.razao = 0
            self.volume = 0
            self.contador_dias = 0
        
    def opera(self):
        period = 45
        c = 0
        s1 = self.dados_var[self.t1]
        s2 = self.dados_var[self.t2]
        for index in range(period, len(self.dados_var)):
            j1 = (s1[c:index]) #seleciona a acao 1 com uma janela com *period* dias 
            j2 = (s2[c:index])

            preco_s1 = self.dados_preco[self.t1][self.dados_var.index == index].values[0]
            preco_s2 = self.dados_preco[self.t2][self.dados_var.index == index].values[0]
            
            zs = self.calcula_zscore_spread(stock1=j1,stock2=j2)
            self.condicoes_operar(zs=zs, gatilho_high = 1.1, preco_s1=preco_s1, preco_s2=preco_s2, index=index)
            c += 1

        

In [4]:
data = pd.read_excel('/home/daniel/Desktop/Projetos/mql_python/Relatório 1/dados_relatorio.xlsx')[1:]
# data = data.drop(['time','close IBOV','close AZUL4','close BBSE3','close BPAC11','close BRDT3','close CRFB3','close CVCB3','close GNDI3','close HAPV3','close IRBR3','close KLBN11','close TAEE11','close VVAR3'],axis=1)
data_var = data.pct_change()[1:600]
data_preco = data[1:600]

TypeError: cannot perform __truediv__ with this index type: DatetimeArray

In [158]:
# aa = pd.DataFrame(np.sum(data_var.isnull()))
# for i in aa.index:
#     print(i,aa[aa.index==i].values[0][0])
#     print('\n')

In [178]:
ltick = data_var.columns.to_list()
ltick_usados = []
for i in range(41,len(ltick)):
    for j in range(i+1,len(ltick)):
        if i <= 50:
            ltick_usados.append([ltick[i],ltick[j]])


In [179]:
doc = {}
ll_par_usado = []
for i in ltick_usados:
    a = (i[0]+' x '+i[1])
    ll_par_usado.append(a)
    doc[a] = Pair_trading(data_var=data_var, data_preco=data_preco, par=i, valor_inicial=1000)
    doc[a].opera()

In [180]:
lista_rentabilidades = []
for i in ll_par_usado:
    lista_rentabilidades.append(doc[i].perm_pct_acumulado)

In [181]:
cleanedList = [x for x in lista_rentabilidades if str(x) != 'nan']
cleanedList = sorted(cleanedList)

In [182]:
for i in cleanedList:
    for k,v in doc.items():
        if v.perm_pct_acumulado == i:
            print(f'{k} {i:.2f}')

close PETR3 x close PETR4 1.42
close MRVE3 x close VIVT4 2.00
close QUAL3 x close VIVT4 2.02
close RENT3 x close VIVT4 2.08
close RAIL3 x close VIVT4 2.15
close QUAL3 x close UGPA3 2.23
close SANB11 x close VIVT4 2.23
close PETR3 x close VIVT4 2.24
close PETR4 x close VIVT4 2.31
close MRVE3 x close UGPA3 2.32
close NTCO3 x close VIVT4 2.34
close PETR4 x close UGPA3 2.40
close PETR3 x close UGPA3 2.44
close MRVE3 x close MULT3 2.45
close MULT3 x close UGPA3 2.47
close RAIL3 x close RENT3 2.48
close RADL3 x close VIVT4 2.48
close MULT3 x close VIVT4 2.49
close RAIL3 x close UGPA3 2.50
close QUAL3 x close WEGE3 2.57
close MRVE3 x close SANB11 2.58
close RENT3 x close UGPA3 2.58
close SANB11 x close UGPA3 2.59
close RAIL3 x close WEGE3 2.60
close NTCO3 x close UGPA3 2.66
close SANB11 x close WEGE3 2.67
close MULT3 x close NTCO3 2.70
close RENT3 x close WEGE3 2.72
close MRVE3 x close NTCO3 2.77
close RAIL3 x close TOTS3 2.79
close RENT3 x close SBSP3 2.80
close RADL3 x close RENT3 2.80
clos

## lista ações do ibov 2017 que deve ser usada 

In [23]:
lttt = 'ABEV3,BBAS3,BBDC3,BBDC4,BBSE3,BRAP4,BRFS3,BRKM5,BRML3,BVMF3,CCRO3,CIEL3,CMIG4,CPFE3,CPLE6,CSAN3,CSNA3,CTIP3,CYRE3,ECOR3,EGIE3,ELET3,EMBR3,ENBR3,EQTL3,ESTC3,FIBR3,GGBR4,GOAU4,HYPE3,ITSA4,ITUB4,JBSS3,KLBN11,KROT3,LAME4,LREN3,MRFG3,MRVE3,MULT3,NATU3,PCAR4,PETR3,PETR4,QUAL3,RADL3,RENT3,RUMO3,SANB11,SBSP3,SMLE3,SUZB5,TIMP3,UGPA3,USIM5,VALE3,VALE5,VIVT4,WEGE3'
lttt = lttt.split(',')
jjj=[]

In [24]:
pend(i[6:])for i in data.columns.to_list():
    jjj.ap

In [25]:
for i in lttt:
    if i not in jjj:
        print(i)

BVMF3
CPLE6
CTIP3
ESTC3
FIBR3
KROT3
NATU3
PCAR4
RUMO3
SMLE3
SUZB5
VALE5


In [19]:
lttt

'ABEV3,BBAS3,BBDC3,BBDC4,BBSE3,BRAP4,BRFS3,BRKM5,BRML3,BVMF3,CCRO3,CIEL3,CMIG4,CPFE3,CPLE6,CSAN3,CSNA3,CTIP3,CYRE3,ECOR3,EGIE3,ELET3,EMBR3,ENBR3,EQTL3,ESTC3,FIBR3,GGBR4,GOAU4,HYPE3.ITSA4.ITUB4,JBSS3,KLBN11,KROT3,LAME4,LREN3,MRFG3,MRVE3,MULT3,NATU3,PCAR4,PETR3,PETR4,QUAL3,RADL3,RENT3,RUMO3,SANB11,SBSP3,SMLE3,SUZB5,TIMP3,UGPA3,USIM5,VALE3,VALE5,VIVT4,WEGE3'

In [26]:
for i in jjj:
    if i not in lttt:
        print(i)


IBOV
AZUL4
B3SA3
BEEF3
BPAC11
BRDT3
BTOW3
COGN3
CRFB3
CVCB3
ELET6
ENGI11
FLRY3
GNDI3
GOLL4
HAPV3
HGTX3
IGTA3
IRBR3
MGLU3
NTCO3
RAIL3
SULA11
SUZB3
TAEE11
TOTS3
VVAR3
YDUQ3


In [27]:
print(lttt)

['ABEV3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BRAP4', 'BRFS3', 'BRKM5', 'BRML3', 'BVMF3', 'CCRO3', 'CIEL3', 'CMIG4', 'CPFE3', 'CPLE6', 'CSAN3', 'CSNA3', 'CTIP3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'EMBR3', 'ENBR3', 'EQTL3', 'ESTC3', 'FIBR3', 'GGBR4', 'GOAU4', 'HYPE3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'KROT3', 'LAME4', 'LREN3', 'MRFG3', 'MRVE3', 'MULT3', 'NATU3', 'PCAR4', 'PETR3', 'PETR4', 'QUAL3', 'RADL3', 'RENT3', 'RUMO3', 'SANB11', 'SBSP3', 'SMLE3', 'SUZB5', 'TIMP3', 'UGPA3', 'USIM5', 'VALE3', 'VALE5', 'VIVT4', 'WEGE3']
